In [ ]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [ ]:
"""Graph zImage size for mainline:omap2plus_defconfig over time"""
import requests
from urlparse import urljoin
import kernelci_api_key

BACKEND_URL = "http://api.kernelci.org"
JOB = 'mainline'
DEFCONFIG = 'omap2plus_defconfig'
DATE_RANGE = 40
STORAGE_SERVER = 'http://storage.kernelci.org'

def invoke():
    headers = {
        "Authorization": kernelci_api_key.getkernelcikey()
    }
    params = {
        "job":JOB,
        "defconfig":DEFCONFIG,
        "date_range": DATE_RANGE
    }
    url = urljoin(BACKEND_URL, "/build")
    response = requests.get(url, params=params, headers=headers)
    return response.content

In [ ]:
content = invoke()
content

In [ ]:
contentjs = json.loads(content)
contentjs['result']

In [ ]:
df = json_normalize(contentjs['result'])
df.head()

In [ ]:
df.columns

In [ ]:
df.kernel_image_size.value_counts()

In [ ]:
df2 = df[[u'_id.$oid',u'git_describe',u'created_on.$date',u'kernel_image_size',u'modules_size']]
df2.head()

In [ ]:
df2['created_on.$date'] = pd.to_datetime(df['created_on.$date'],unit='ms')
df2.head()

In [ ]:
df2.columns = ['build_id','git_describe','created_on','kernel_image_size','modules_size']

In [ ]:
df2.index = df2.created_on
df2.head()

In [ ]:
df2.loc[df2.index[1]]['modules_size']

In [ ]:
import urllib
for x in range(0,len(df2)):
    fileloc = STORAGE_SERVER + '/' + JOB + '/' + df2.iloc[x]['git_describe'] + '/arm-' + DEFCONFIG + '/zImage'
    filesize = urllib.urlopen(fileloc).info()['Content-Length']
    df2.set_value(df2.index[x], 'kernel_image_size', int(filesize))
    print x
    print filesize

In [ ]:
df2.kernel_image_size = df2.kernel_image_size.astype(int)

In [ ]:
df2.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df2.kernel_image_size.describe()['min']

In [ ]:
df2

In [ ]:
df2['2015-09-24 ':]

In [ ]:
df2.kernel_image_size.sort_index().plot(kind='line')
plt.ylim(3570000,3590000)